<a href="https://colab.research.google.com/github/k-nithin/kenco_takehome/blob/main/Kenco_Take_Home_Nithin_Kakani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [167]:
import pandas as pd

In [168]:
def parse_data(data):
  def space_remover(dataframe):
    
      for i in dataframe.columns:
          # checking datatype of each columns
          if dataframe[i].dtype == 'object':
        
              # removing extra spaces in all columns
              dataframe[i] = dataframe[i].map(str.strip).str.replace(r'\s+', ' ')
          else:
              pass

  def DaysWorked_cleaner(dataframe):

    # Changing records to title case
    dataframe["DaysWorked"] = dataframe["DaysWorked"].str.title()

    # Splitting DaysWorked column into new df for further processing
    df = dataframe['DaysWorked'].str.split(' ',expand=True)

    # Filling null values
    df = df.fillna("")

    # Getting first 3 letter from all days
    for i in range(len(df.columns)):
      df[i] = df[i].astype(str).str[:3]

    
    # merge all days into one column
    df['ColumnA'] = df[df.columns[0:5]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1).str.strip(',')

    # replace old column with cleaned column
    dataframe.DaysWorked = df.ColumnA
    del df



  def time_format(dataframe,columnname):

    # Formating time for date columns
    for i in dataframe.columns:
          # checking datatype of each columns
          if dataframe[i].dtype == 'object':
              dataframe[columnname] = dataframe[columnname].astype('datetime64[ns]').dt.strftime('%H:%M')

  # Calling fucntions created on required columns
  space_remover(data)
  DaysWorked_cleaner(data)
  time_format(data,"Begin")
  time_format(data,"LunchBegin")
  time_format(data,"LunchEnd")
  time_format(data,"End")
  return data

In [175]:
from datetime import datetime
if __name__ == "__main__":
    file = 'https://raw.githubusercontent.com/k-nithin/kenco_takehome/a0fdccec196e392943cff58b0613fe2efe6688ee/TakeHome.csv'
    
    data = pd.read_csv(file,skipinitialspace = True,header=None,sep='/',names=["BrokerName","DaysWorked","Begin","LunchBegin","LunchEnd","End"])
    print(data)
    modified_data = parse_data(data)
    print(modified_data)

    BrokerName                           DaysWorked   Begin LunchBegin  \
0    Broker A          Mon Tue WEDNESDAY Thu Fri     05:00      10:00    
1    Broker B            Tue Wed Thursday Fri Sat    08:00      12:30    
2    Broker D             Mon  Wednesday  Fri  Sun   06:30      11:30    
3    Broker E              Tue Wed Thu Fri Sat Sun   07:00      11:00    
4    Broker F           Mon Tue Wed Thursday Fri     15:00      21:00    
5    Broker G                   Mon  Wed Thu  Sat    09:30      13:00    
6    Broker H                      Mon    Fri Sat    10:00      14:00    
7    Broker I                Mon Tue Wednesday       04:00      10:00    
8    Broker J            Tue Wed Thursday Fri Sat    06:45      10:15    
9    Broker K          Mon Tue Wednesday Thu Fri     07:30      12:30    
10   Broker L            Tue Wed Thursday Fri Sat    05:00      09:00    
11   Broker N                     Mon Tue wEDN       13:00      19:00    
12   Broker O                   Mon Tu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':



# Please answer the following questions alongside the dimension creation:
# 1.   What method was used to force uniformity on the different formatting within the DaysWorked column? 


I've created a function (DaysWorked_cleaner) which will first change all the records into title case and then splitted them into new dataframe and sliced first 3 letters to get them all in short day of the week format.
Upon investigating I found data with incorrect spelling, so this method works efficently assuming first 3 letters in the given data is correct.




---



# 2) What method could be used to parse this data into separate columns using SQL?

We can use below method to parse given csv file in SQL.

```
CREATE TABLE brokerschedule (BrokerName VARCHAR(40), DaysWorked VARCHAR(40),Begin DATETIME,LunchBegin DATETIME,LunchEnd DATETIME,End DATETIME)
BULK INSERT TAKEHOME.CSV FROM '\files\TakeHome.csv' 
WITH ( FIELDTERMINATOR = '/', ROWTERMINATOR = '\n', FirstRow=2)
```



---




# 3) What method could be used to parse this data into separate columns using Python?


I've used pandas read_csv function and utilised option sep = '/' to parse the given csv file.



---


# 4) Is there a better column structure to create a dimension table from the data in the CSV file? For example, instead of having a DaysWorked column, would it be better to have a column for each day of the week. Please elaborate on your answer.



Yes having a column for each day would be more helpful in further data processing and analytical stand point.
Pivoting column for each day by maintaining boolean value in data would help us understand the brokers schedule more easily by running queries over.
In current columns looking for broker that's available on 2 days would be difficult as we must use 'like' or regex to find the answer increasing computation power on huge data.




---




# 5) What are features you could derive from this table for feature engineering within a machine learning model?


We can derive broker characteristics like relationship on working days and start and end time.

We can derive maximum brokers available at a given time and day which helps to utilize the workforce and distribute the workload.

We can derive which day usually brokers take the longest lunch break.

We can derive which day usally maximum broker works.

